<a href="https://colab.research.google.com/github/mequanent/Bioinformatics-hws/blob/main/hw4_110761504.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# Clone the repository first using the following command
!npx degit https://github.com/mequanent/Bioinformatics-hws -f

npx: installed 1 in 2.825s
> destination directory is not empty. Using --force, continuing
> cloned mequanent/Bioinformatics-hws#HEAD


In [6]:
pip install Bio -q 

# HW4: Global and Local Alignments with Affine Gap Penalties

In [7]:
import numpy as np
import pandas as pd

from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

In [8]:


seq_name = []
sequence = []

# for seq_record in SeqIO.parse("test3.fasta", "fasta"):
for seq_record in SeqIO.parse("pro4_2.fasta", "fasta"): # 
  seq_name.append(seq_record.id)
  sequence.append(seq_record.seq)

seq1 = sequence[0]
seq2 = sequence[1]

print("+----- Given Sequences with their respective lengthes ---------+")
print(len(seq1), seq1)
print(len(seq2), seq2)
print("****************************************************************")
gop= -10
gex = -2
score = "pam250.txt" # score from command line args
aln = 'global' # Alignment choice argument from command line interface

col = '-' + seq1 # This if for simplicity to avoid repeated use of (len(seq1) + 1)
row = '-' + seq2 # For the same reason

def preprocess_score(score):
  file = open(score, 'r')
  lines = file.readlines()
  file.close()
  columns = []
  mut = []
  for line in lines:
    if line.startswith("#") or line.startswith(" "):
      continue
    columns.append(line[0]) # Amino acid alphabets stored for row & column indexing
    line = line[1:].strip().split() # remove the letters from left and the spaces
    mut.append(line)
  mut = np.array(mut).astype(int) # change the string numbers into type int
  return pd.DataFrame(mut, index = columns, columns = columns)

score = preprocess_score(score) # score made a data frame to access its values  
                  # with alphabet indices

def pairwise(x, y):
  pair_sum = 0
  x_prev = '' # x_prev and y_prev are used to store immediate previous alphabets
  y_prev = '' # so that extension gaps will be identified
  for i in range(len(x)): # Considering that both x and y have the same length
    x_prev = (x_prev == '-' and x[i] == '-') # status of extension gap in the first paired sequence
    y_prev = (y_prev == '-' and y[i] == '-') # status of extension gap in second paired sequence
    curr = (x[i] == '-' or y[i] == '-') # status of open gap in the paired sequence
    if x_prev or y_prev: # extension gap value will be taken if holds
      pair_sum = pair_sum + gex
    elif curr: # open gap value will be taken if holds
      pair_sum = pair_sum + gop
    else: # score matrix value will be taken otherwise
      pair_sum = pair_sum + score.loc[x[i]][y[i]] 
    x_prev = x[i]
    y_prev = y[i]
  return pair_sum

def createMatrices():
  M = np.zeros((len(row), len(col))) # score matrix for global alignment 
  Ix = np.zeros((len(row), len(col)))
  Iy = np.zeros((len(row), len(col)))

  traceM = np.zeros((len(row), len(col)), dtype = object)
  traceIx = np.zeros((len(row), len(col)), dtype = object)
  traceIy = np.zeros((len(row), len(col)), dtype = object)

  return M, Ix, Iy, traceM, traceIx, traceIy

# ***************** Global Alignment Section *************************
def globalAlignment():
  # Filling score matrices and keeping track of the direction for each cell
  M, Ix, Iy, traceM, traceIx, traceIy = createMatrices()

  # Initializing the first column and first row
  Ix[0][0] = Iy[0][0] = gop
  for i in range(1, len(row)):
    M[i][0] = Iy[i][0] = np.NINF
    Ix[i][0] = gop + i * gex 
  for j in range(1, len(col)):
    M[0][j] = Ix[0][j] = np.NINF
    Iy[0][j] = gop + j * gex

  for i in range(1, len(row)):
    for j in range(1, len(col)):
      mat = score.loc[row[i]][col[j]]
      M[i][j] = max((M[i-1][j-1] + mat), (Ix[i-1][j-1] + mat), (Iy[i-1][j-1] + mat)) 
      Ix[i,j] = max((M[i - 1, j] + gop), (Ix[i - 1, j] + gex))
      Iy[i,j] = max((M[i, j - 1] + gop), (Iy[i, j - 1] + gex))

      if M[i][j] == (Ix[i-1][j-1] + mat):
        traceM[i][j] = 'x'
      elif M[i][j] == (Iy[i-1][j-1] + mat):
        traceM[i][j] = 'y'
      else:
        traceM[i][j] = 'm'

      if Ix[i][j] == (Ix[i - 1, j] + gex):
        traceIx[i][j] = 'x'
      else:
        traceIx[i][j] = 'm'
      
      if Iy[i][j] == (Iy[i, j - 1] + gex):
        traceIy[i][j] = 'y'
      else:
        traceIy[i][j] = 'm'   

# Trace back section for global alignment
  j = len(seq1)
  i = len(seq2)
  aln1 = ''
  aln2 = ''
  mxm = max(M[i][j], Ix[i][j], Iy[i][j])
  tracer = ''
  if mxm == M[i][j]:
    tracer = 'm'
  elif mxm == Ix[i][j]:
    tracer = 'x'
  else:
    tracer = 'y'
    
  while (i > 0 and j > 0):
    if tracer == 'm':
      aln1 = col[j] + aln1
      aln2 = row[i] + aln2
      if traceM[i][j] == 'm':
        tracer = 'm'
      elif traceM[i][j] == 'x':
        tracer = 'x'
      elif traceM[i][j] == 'y':
        tracer = 'y'
      i, j = i - 1, j - 1
      continue
    
    elif tracer == 'x':
      aln1 = '-' + aln1
      aln2 = row[i] + aln2
      if traceIx[i][j] == 'm':
        tracer = 'm'    
      elif traceIx[i][j] == 'x':
        tracer = 'x'
      i = i - 1
      continue
      
    elif tracer == 'y':
      aln1 = col[j] + aln1
      aln2 = '-' + aln2
      if traceIy[i][j] == 'm':
        tracer = 'm'
      elif traceIy[i][j] == 'y':
        tracer = 'y'
      j = j - 1
    else:
      break
      
  return aln1, aln2
# --------------------------------------------
# ************* End of Global Alignment ************************************

def isbest(a, b):
  if float(a) > float(b):
    return a
  else:
    return b
# -------------------------------------------------------------------
# Local Alignment Section 
def localAlignment():
  bestScore = 0.0
  # Filling score matrices and keeping track of the direction for each cell
  M, Ix, Iy, traceM, traceIx, traceIy = createMatrices()

  for i in range(1, len(row)):
    for j in range(1, len(col)):
      mat = score.loc[row[i]][col[j]]
      M[i][j] = max((M[i-1][j-1] + mat), (Ix[i-1][j-1] + mat), (Iy[i-1][j-1] + mat), 0.0) 
      Ix[i,j] = max((M[i - 1, j] + gop), (Ix[i - 1, j] + gex), 0.0)
      Iy[i,j] = max((M[i, j - 1] + gop), (Iy[i, j - 1] + gex), 0.0)

      if M[i][j] == (Ix[i-1][j-1] + mat):
        traceM[i][j] = 'x'
        bestScore = isbest(M[i][j], bestScore)
      elif M[i][j] == (Iy[i-1][j-1] + mat):
        traceM[i][j] = 'y'
        bestScore = isbest(M[i][j], bestScore)
      elif M[i][j] == (M[i-1][j-1] + mat):
        traceM[i][j] = 'm'
        bestScore = isbest(M[i][j], bestScore)

      if Ix[i][j] == (Ix[i - 1, j] + gex):
        traceIx[i][j] = 'x'
        bestScore = isbest(Ix[i][j], bestScore)
      elif Ix[i][j] == (M[i - 1, j] + gop):
        traceIx[i][j] = 'm'
        bestScore = isbest(Ix[i][j], bestScore)
      
      if Iy[i][j] == (Iy[i, j - 1] + gex):
        traceIy[i][j] = 'y'
        bestScore = isbest(Iy[i][j], bestScore)
      elif Iy[i][j] == (M[i, j - 1] + gop):
        traceIy[i][j] = 'm'   
        bestScore = isbest(Iy[i][j], bestScore)

# Trace back section for local alignment
  r, c = M.shape # row and col lengths to track best score locations
  bestLoc = [] # list of best score cells
  pairs = [] # list of local pairs
  
  # Find all locations of the best score from the local alignment score matrix
  for i in range(1, r):
    for j in range(1, c):
      if M[i][j] == bestScore:
        bestLoc.append((i, j))
      if Ix[i][j] == bestScore:
        bestLoc.append((i, j))
      if Iy[i][j] == bestScore:
        bestLoc.append((i, j))
  
  for k in range(len(bestLoc)): # loop over all optimal local alignments
    i, j = bestLoc[k]
    aln1 = ''
    aln2 = ''
    mxm = max(M[i][j], Ix[i][j], Iy[i][j], 0)
    tracer = ''
    if mxm == M[i][j]:
      tracer = 'm'
    elif mxm == Ix[i][j]:
      tracer = 'x'
    else:
      tracer = 'y'
      
    while (i > 0 and j > 0):# and tracer != ''):
      if tracer == 'm' and M[i][j]>0:
        aln1 = col[j] + aln1
        aln2 = row[i] + aln2
        if traceM[i][j] == 'm':
          tracer = 'm'
        elif traceM[i][j] == 'x':
          tracer = 'x'
        elif traceM[i][j] == 'y':
          tracer = 'y'
        i, j = i - 1, j - 1
        continue
      
      elif tracer == 'x' and Ix[i][j] > 0:
        aln1 = '-' + aln1
        aln2 = row[i] + aln2
        if traceIx[i][j] == 'm':
          tracer = 'm'    
        elif traceIx[i][j] == 'x':
          tracer = 'x'
        i = i - 1
        continue
        
      elif tracer == 'y' and Iy[i][j] > 0:
        aln1 = col[j] + aln1
        aln2 = '-' + aln2
        if traceIy[i][j] == 'm':
          tracer = 'm'
        elif traceIy[i][j] == 'y':
          tracer = 'y'
        j = j - 1
      else:
        break
    pairs.append((aln1, aln2))
  return pairs  
# End of Local Alignment
# --------------------------------------------

def showMatched(x, y): # A function to help proper display of the alignments
  between = ''
  for i in range(len(x)):
    if x[i] == y[i]:
      between = between + '|'
    elif x[i] == '-' or y[i] == '-':
      between = between + ' '
    else:
      between = between + '.'
  return between

def displayAlignment(x, y): # A function to display the aligned sequences with their SoP score
  print(x)
  print(showMatched(x, y))
  print(y)
  print("Sum of pairs (SoP) score for the {a} alignment is: {b}".format(a = aln, b = pairwise(x, y)))

# The SoP value verifies the correctness of our alignment if it becomes
# equal to the right bottom value for global and to the best score for local.

# --------------------------------------------
# **** Do the selected alignment by calling corresponding functions ***********
print("+----- You wanted {} alignment and here is the result -----+".format(aln))  
alignments = []
al1 = al2 = ''
if aln == 'global':
  aln1, aln2 = globalAlignment()
  al1, al2 = aln1, aln2
  displayAlignment(aln1, aln2)
  alignments.append(SeqRecord(Seq(aln1), id="protien 1"))
  alignments.append(SeqRecord(Seq(aln2), id="protien 2"))
elif aln == 'local':
  j = 0
  for i in localAlignment():
    print("\n****** Local Alignment: {} ******** ".format(j))
    displayAlignment(i[0], i[1])
    alignments.append(SeqRecord(Seq(i[0]), id=seq_name[0], description=":|protien 1"))
    alignments.append(SeqRecord(Seq(i[1]), id=seq_name[1], description=":|protien 2"))
    j += 1
else:
  print("Please select the alignment type.")

SeqIO.write(alignments, "result.fasta", "fasta")

+----- Given Sequences with their respective lengthes ---------+
636 MRTLSPARPHRTPSKSLPPCSTPACSPRSSASTVFRSLSLLQSWILTLSLHLHRQRQLARVNCPVTMDFKRLIADLSPAQRAEVLASLIPMMSTTELQLARQLIRTPSGPITVPVAPVCHAHATLSMSWPGPAPSTVPRSKSPDCACACTCTDGTPRPQFRDLPDEILLKIVAYLRPQSLARLARTCHRFAVLIRDSLLWKNMLAKYRGQGYAAYRPHEDKLRVLRSTTTPPAPIPIVYSPEKALVRRDSGVARSPTAAAALFGTSPDGSRNGSLPARNGHQEIPKSLGSFWRDEFRRGYLTHLNWKRGRCRIAKVLRRPGSLHADMDPDRNLIASIDADGSKTWDVMTGETTTQLPSQLGSFTTIKFFQHYTLTGSTNCSVRVWDHATGQLVRAFTGVHTREVSAIDADQATVASGSEDTVIWTWELLSGKPLQYFRGHTGPICSLKLDMARGILLSGSADTTVRVWDVAPTLMAPDNPTTPIVQLGKCRAVLRGHQNEVFCVALASNLVVSGSGDCRIKVWSLETKSLLRTLEGHTAAVIALQVDNNKIISGSADKTIKVWDLHNPAQALYSLSDQHTSGLLALRFNDKQLFASDMASTIVVYDFEHEDDKQDTDPRAPPSPSSSTGSMHFCTY
536 MTPWDTVAPQTLSLPRERRIALLAALLRDASDSDLLLAQAQAVATLLPEIPTASLNGDGFQHKRLPPLLMDLPDELLIHVLARIATPADLAATARTSVRMAAIVRDNHLWRDLVVRLGAEGFGAYPRPPPVTRRDSGLLAVVVASGPSETKTTKVPRSMVRSPWHSEFVKGYKTAMNWKRGACTVSRVVRRPGSLHADFDVGRSLVVSFTPSSSITWNLETGALHSQISSNPASAITTIRLTADHILTATSDAAIVVHDRATGVPLVRLRADRIPTAPQSEYGAVDADGT

2

In [ ]:
print("+----- You wanted {} alignment and here is the result -----+".format(aln))  
alignments = []
al1 = al2 = ''
if aln == 'global':
  aln1, aln2 = globalAlignment()
  al1, al2 = aln1, aln2
  displayAlignment(aln1, aln2)
  alignments.append(SeqRecord(Seq(aln1), id="protien 1"))
  alignments.append(SeqRecord(Seq(aln2), id="protien 2"))
elif aln == 'local':
  j = 0
  for i in localAlignment():
    print("\n****** Local Alignment: {} ******** ".format(j))
    al1, al2 = i[0], i[1]
    displayAlignment(al1, al2)
    alignments.append(SeqRecord(Seq(al1), id=seq_name[0], description=":|protien 1"))
    alignments.append(SeqRecord(Seq(al2), id=seq_name[1], description=":|protien 2"))
    j += 1
else:
  print("Please select the alignment type.")

+----- You wanted global alignment and here is the result -----+
MRTLSPARPHRTPSKSLPPCSTPACSPRSSASTVFRSLSLLQSWILTLSLHLHRQRQLARVNCPVTMDFKRLIADLSPAQRAEVLASLIPMMSTTELQLARQLIRTPSGPITVPVAPVCHAHATLSMSWPGPAPSTVPRSKSPDCACACTCTDGTPRPQFRDLPDEILLKIVAYL-RPQSLARLARTCHRFAVLIRDSLLWKNMLAKYRGQGYAAYRPHEDKLRVLRSTTTPPAPIPIVYSPEKALVRRDSGVARSPTAAAALFGTSPDGSRNGSLPARNGHQEIPKSLGSFWRDEFRRGYLTHLNWKRGRCRIAKVLRRPGSLHADMDPDRNLIASIDADGSKTWDVMTGETTTQLPSQLGS-FTTIKFFQHYTLTGSTNCSVRVWDHATG-QLVRA----FTGVHTREVSAIDADQATVASGSEDTVIWTWEL---LS------GKPLQ--------------YFRGHTGPICSLKLDMARGILLSGSADTTVRVWDVAPTLM---APDNPTTPIVQLGKCRAVLRGHQNEVFCVALASNLVVSGSGDCRIKVWSL-----ETKS---------LLRTLEGHTAAVIALQVDNNKIISGSADKTIKVWDLHN----PAQALYSLSDQHTSGLLALRFNDKQLFASDMASTIVVYDFEHEDDKQDTDPRAPPSPSSSTGSMHFCTY
|          ||.....|                            ..|.|.|.|..|                        .||.|....|...|.||.    ........|..|.    |.|.       .........|              |...|||||.|....|.. .|..||..|||..|.|...||..||..........|..||                |.|.|..        

# End of HW4

In [26]:
sequence2 = []
seq_new = []
seq_mine = []

# for seq_record in SeqIO.parse("test3.fasta", "fasta"): pro4_ebi_result.fasta
for seq_record in SeqIO.parse("pro4_2_result.fasta", "fasta"): # 
  sequence2.append(seq_record.seq)

for seq_record in SeqIO.parse("pro4_ebi_result.fasta", "fasta"): # 
  seq_new.append(seq_record.seq)

for seq_record in SeqIO.parse("pro4_mine.fasta", "fasta"): # 
  seq_mine.append(seq_record.seq)


In [27]:
#[i for i, j in zip(a, b) if i == j]
seq11, seq22 = sequence2[0], sequence2[1]

sq1, sq2 = seq_new[0], seq_new[1]
s1, s2 = seq_mine[0], seq_mine[1]

print(seq11)
print(sq1)
print(s1)
print()
print(seq22)
print(sq2)
print(s2)
#al1

MRTLSPARPHRTPSKSLPPCSTPACSPRSSASTVFRSLSLLQSW--ILTLSLHLHRQRQLARVNCPVTMDFKRLIADLSPAQRAEVLASLIPMMSTTELQLARQLIRTPSGPITVPVAPVCHAHATLSMSWPGPAPSTVPRSKSPDCACACTCTDGTPRPQFRDLPDEILLKIVAYL-RPQSLARLARTCHRFAVLIRDSLLWKNMLAKYRGQGYAAYRPHEDKLRVLRSTTTPPAPIPIVYSPEKALVRRDSGVARSPTAAAALFGTSPDGSRNGSLPARNGHQEIPKSLGSFWRDEFRRGYLTHLNWKRGRCRIAKVLRRPGSLHADMDPDRNLIASIDADGSKTWDVMTGETTTQLPSQ-LGSFTTIKFFQHYTLTGSTNCSVRVWDHATG-QLVRAFTG----VHTREVSAIDADQATVASGSEDTVIWTWE----LL-----SGKPL--------------QYFRGHTGPICSLKLDMARGILLSGSADTTVRVWDVAPTL---MAPDNPTTPIVQLGKCRAVLRGHQNEVFCVALASNLVVSGSGDCRIKVW--------------SLETKSLLRTLEGHTAAVIALQVDNNKIISGSADKTIKVWDLH----NPAQALYSLSDQHTSGLLALRFNDKQLFASDMASTIVVYDFEHEDDKQDTDPRAPPSPSSSTGSMHFCTY
MRTLSPARPHRTPSKSLPPCSTPACSPRSSASTVFRSLSLLQSWILTLSLHLHRQRQLARVNCPVTMDFKRLIADLSPAQRAEVLASLIPMMSTTELQLARQLIRTPSGPITVPVAPVCHAHATLSMSWPGPAPSTVPRSKSPDCACACTCTDGTPRPQFRDLPDEILLKIVAYL-RPQSLARLARTCHRFAVLIRDSLLWKNMLAKYRGQGYAAYRPHEDKLRVLRSTTTPPAPIPIVYSPEKALVRRDSGVARSPTAAAALFGTSPDGSRNGSLPARNGHQEIPKSLGSFWRDEFRRGYLTHLNWKRG

In [9]:
pairwise(sq1, sq2)

473

# Second Option

In [ ]:
# Second Option
def globalAlignment2():
  M, Ix, Iy, traceM, traceIx, traceIy = createMatrices()

  # Initializing the first column and first row
  Ix[0][0] = Iy[0][0] = gop
  for i in range(1, len(row)):
    M[i][0] = Iy[i][0] = np.NINF
    Ix[i][0] = gop + i * gex 
  for j in range(1, len(col)):
    M[0][j] = Ix[0][j] = np.NINF
    Iy[0][j] = gop + j * gex

  for i in range(1, len(row)):
    for j in range(1, len(col)):
      mat = score.loc[row[i]][col[j]]
      M[i][j] = max((M[i-1][j-1] + mat), (Ix[i-1][j-1] + mat), (Iy[i-1][j-1] + mat)) 
      Ix[i,j] = max((M[i - 1, j] + gop), (Ix[i - 1, j] + gex))
      Iy[i,j] = max((M[i, j - 1] + gop), (Iy[i, j - 1] + gex))

      if M[i][j] == (M[i-1][j-1] + mat):
        traceM[i][j] = 'm'
      elif M[i][j] == (Ix[i-1][j-1] + mat):
        traceM[i][j] = 'x'
      elif M[i][j] == (Iy[i-1][j-1] + mat):
        traceM[i][j] = 'y'

      if Ix[i][j] == (M[i-1][j] + gop):
        traceIx[i][j] = 'm'
      else:
        traceIx[i][j] = 'x'
      
      if Iy[i][j] == (M[i][j-1] + gop):
        traceIy[i][j] = 'm'
      else:
        traceIy[i][j] = 'y'    

# Trace back section for global alignment
#def traceBack():
  j = len(seq1)
  i = len(seq2)
  aln1 = ''
  aln2 = ''
  mxm = max(M[i][j], Ix[i][j], Iy[i][j])
  tracer = ''
  if mxm == M[i][j]:
    tracer = 'm'
  elif mxm == Ix[i][j]:
    tracer = 'x'
  else:
    tracer = 'y'
    
  while (i > 0 and j > 0):
    if tracer == 'm':
      aln1 = col[j] + aln1
      aln2 = row[i] + aln2
      #i, j = i - 1, j - 1
      if traceM[i][j] == 'm':
        tracer = 'm'
      elif traceM[i][j] == 'x':
        tracer = 'x'
      elif traceM[i][j] == 'y':
        tracer = 'y'
      i, j = i - 1, j - 1
      continue
    
    elif tracer == 'x':
      aln1 = '-' + aln1
      aln2 = row[i] + aln2
      #i = i - 1
      if traceIx[i][j] == 'm':
        tracer = 'm'    
      elif traceIx[i][j] == 'x':
        tracer = 'x'
      i = i - 1
      continue
      
    elif tracer == 'y':
      aln1 = col[j] + aln1
      aln2 = '-' + aln2
      if traceIy[i][j] == 'm':
        tracer = 'm'
      elif traceIy[i][j] == 'y':
        tracer = 'y'
      j = j - 1
    else:
      break
      
  return aln1, aln2
# End of Trace Back for the Global Alignment
# --------------------------------------------
globalAlignment2()

('MRTLSPARPHRTPSKSLPPCSTPACSPRSSASTVFRSLSLLQSWILTLSLHLHRQRQLARVNCPVTMDFKRLIADLSPAQRAEVLASLIPMMSTTELQLARQLIRTPSGPITVPVAPVCHAHATLSMSWPGPAPSTVPRSKSPDCACACTCTDGTPRPQFRDLPDEILLKIVAYL-RPQSLARLARTCHRFAVLIRDSLLWKNMLAKYRGQGYAAYRPHEDKLRVLRSTTTPPAPIPIVYSPEKALVRRDSGVARSPTAAAALFGTSPDGSRNGSLPARNGHQEIPKSLGSFWRDEFRRGYLTHLNWKRGRCRIAKVLRRPGSLHADMDPDRNLIASIDADGSKTWDVMTGETTTQLPSQ-LGSFTTIKFFQHYTLTGSTNCSVRVWDHATG-QLVR----AFTGVHTREVSAIDADQATVASGSEDTVIWTWEL---LSGK--------------------PLQYFRGHTGPICSLKLDMARGILLSGSADTTVRVWDVAPTLM---APDNPTTPIVQLGKCRAVLRGHQNEVFCVALASNLVVSGSGDCRIKVWSL--------------ETKSLLRTLEGHTAAVIALQVDNNKIISGSADKTIKVWDLH----NPAQALYSLSDQHTSGLLALRFNDKQLFASDMASTIVVYDFEHEDDKQDTDPRAPPSPSSSTGSMHFCTY',
 'M----------TPWDTVAP----------------------------QTLSLPRER------------------------RIALLAALLRDASDSDLLLA----QAQAVATLLPEIPT----ASLN-------GDGFQHKRLP--------------PLLMDLPDELLIHVLARIATPADLAATARTSVRMAAIVRDNHLWRDLVVRLGAEGFGAY----------------PRPPPV--------TRRDSGL------LAVVVASGPSETKTTKVP-----RSMVRSP---WHSEFVKGYKTAMN

In [ ]:
print(sq2)

M----------TPWDTVAP----------------------------QTLSLPRERRIA------------------------LLAALLRDASDSDLLLAQ----------AQAVATLLPEIPTASLNGDGFQHKRLP-------------------PLLMDLPDELLIHVLARIATPADLAATARTSVRMAAIVRDNHLWRDLVVRLGAEGFGAY----------------PRPPPVT--------RRDSGL------LAVVVASGPSETKTTKVPRSMVRSP--------WHSEFVKGYKTAMNWKRGACTVSRVVRRPGSLHADFDVGRSLVVSFTPSSSITWNLETGALHSQISSNPASAITTIRLTADHILTATSDAAIVVHDRATGVPLVRLRADRIPTAPQSEYGAVDADGTFCVAGAESGMVTAWDISTALASNSRHKGSPVPPATTAQVHELTPSACLVGHTTSITAVACARDSGLVAAGALDGSIRVWRVETKVSKHGNGDEQQHQQQQLVPIATALGAHPGGVFAMCIQGTHLITGGGDNSLKLWHLPQHHQSSKGAMIAADQLRLQRTFTGHTAPVICVAADSHKIVSGGADRSIKVWAMHATGSAAGTALYSLRDRHEAGLHAVRFDSGKLIASDFTSAVVVYDFEEPSE----------------------TV


In [ ]:
def localAlignment2():
  bestScore = 0.0
  # Filling score matrices and keeping track of the direction for each cell
  M, Ix, Iy, traceM, traceIx, traceIy = createMatrices()

  for i in range(1, len(row)):
    for j in range(1, len(col)):
      mat = score.loc[row[i]][col[j]]
      M[i][j] = max((M[i-1][j-1] + mat), (Ix[i-1][j-1] + mat), (Iy[i-1][j-1] + mat), 0.0) 
      Ix[i,j] = max((M[i - 1, j] + gop), (Ix[i - 1, j] + gex), 0.0)
      Iy[i,j] = max((M[i, j - 1] + gop), (Iy[i, j - 1] + gex), 0.0)

      if M[i][j] == (M[i-1][j-1] + mat):
        traceM[i][j] = 'm'
        bestScore = isbest(M[i][j], bestScore)
      elif M[i][j] == (Ix[i-1][j-1] + mat):
        traceM[i][j] = 'x'
        bestScore = isbest(M[i][j], bestScore)
      elif M[i][j] == (Iy[i-1][j-1] + mat):
        traceM[i][j] = 'y'
        bestScore = isbest(M[i][j], bestScore)
      

      if Ix[i][j] == (Ix[i - 1, j] + gex):
        traceIx[i][j] = 'x'
        bestScore = isbest(Ix[i][j], bestScore)
      elif Ix[i][j] == (M[i - 1, j] + gop):
        traceIx[i][j] = 'm'
        bestScore = isbest(Ix[i][j], bestScore)
      
      if Iy[i][j] == (Iy[i, j - 1] + gex):
        traceIy[i][j] = 'y'
        bestScore = isbest(Iy[i][j], bestScore)
      elif Iy[i][j] == (M[i, j - 1] + gop):
        traceIy[i][j] = 'm'   
        bestScore = isbest(Iy[i][j], bestScore)

# Trace back section for local alignment
  r, c = M.shape # row and col lengths to track best score locations
  bestLoc = [] # list of best score cells
  pairs = [] # list of local pairs
  
  # Find all locations of the best score from the local alignment score matrix
  for i in range(1, r):
    for j in range(1, c):
      if M[i][j] == bestScore:
        bestLoc.append((i, j))
      if Ix[i][j] == bestScore:
        bestLoc.append((i, j))
      if Iy[i][j] == bestScore:
        bestLoc.append((i, j))
  
  for k in range(len(bestLoc)): # loop over all optimal local alignments
    i, j = bestLoc[k]
    aln1 = ''
    aln2 = ''
    mxm = max(M[i][j], Ix[i][j], Iy[i][j], 0)
    tracer = ''
    if mxm == M[i][j]:
      tracer = 'm'
    elif mxm == Ix[i][j]:
      tracer = 'x'
    else:
      tracer = 'y'
      
    while (i > 0 and j > 0):# and tracer != ''):
      if tracer == 'm' and M[i][j]>0:
        aln1 = col[j] + aln1
        aln2 = row[i] + aln2
        if traceM[i][j] == 'm':
          tracer = 'm'
        elif traceM[i][j] == 'x':
          tracer = 'x'
        elif traceM[i][j] == 'y':
          tracer = 'y'
        i, j = i - 1, j - 1
        continue
      
      elif tracer == 'x' and Ix[i][j] > 0:
        aln1 = '-' + aln1
        aln2 = row[i] + aln2
        if traceIx[i][j] == 'm':
          tracer = 'm'    
        elif traceIx[i][j] == 'x':
          tracer = 'x'
        i = i - 1
        continue
        
      elif tracer == 'y' and Iy[i][j] > 0:
        aln1 = col[j] + aln1
        aln2 = '-' + aln2
        if traceIy[i][j] == 'm':
          tracer = 'm'
        elif traceIy[i][j] == 'y':
          tracer = 'y'
        j = j - 1
      else:
        break
    pairs.append((aln1, aln2))
  return pairs  

# Third Option globalAlignment3()

In [10]:

def globalAlignment3():
  M = np.zeros((len(row), len(col))) 
  Ix = np.zeros((len(row), len(col)))
  Iy = np.zeros((len(row), len(col)))

  traceM = np.zeros((len(row), len(col)), dtype = object)
  traceIx = np.zeros((len(row), len(col)), dtype = object)
  traceIy = np.zeros((len(row), len(col)), dtype = object)

  Ix[0][0] = Iy[0][0] = gop
  for i in range(1, len(row)):
    M[i][0] = Iy[i][0] = np.NINF
    Ix[i][0] = gop + i * gex 
  for j in range(1, len(col)):
    M[0][j] = Ix[0][j] = np.NINF
    Iy[0][j] = gop + j * gex

  for i in range(1, len(row)):
    for j in range(1, len(col)):
      mat = score.loc[row[i]][col[j]]
      M[i][j] = max((M[i-1][j-1] + mat), (Ix[i-1][j-1] + mat), (Iy[i-1][j-1] + mat)) 
      Ix[i,j] = max((M[i - 1, j] + gop), (Ix[i - 1, j] + gex))
      Iy[i,j] = max((M[i, j - 1] + gop), (Iy[i, j - 1] + gex))

      if M[i][j] == (Ix[i-1][j-1] + mat):
        traceM[i][j] = 'x'
      elif M[i][j] == (Iy[i-1][j-1] + mat):
        traceM[i][j] = 'y'
      else:
        traceM[i][j] = 'm'

      if Ix[i][j] == (Ix[i - 1, j] + gex):
        traceIx[i][j] = 'x'
      else:
        traceIx[i][j] = 'm'
      
      if Iy[i][j] == (Iy[i, j - 1] + gex):
        traceIy[i][j] = 'y'
      else:
        traceIy[i][j] = 'm'   

# Trace back  for global 
  j = len(seq1)
  i = len(seq2)
  aln1 = ''
  aln2 = ''
  mxm = max(M[i][j], Ix[i][j], Iy[i][j])
  tracer = ''
  if mxm == M[i][j]:
    tracer = 'm'
  elif mxm == Ix[i][j]:
    tracer = 'x'
  else:
    tracer = 'y'
    
  while (i > 0 and j > 0):
    if tracer == 'm':
      aln1 = col[j] + aln1
      aln2 = row[i] + aln2
      if traceM[i][j] == 'm':
        tracer = 'm'
      elif traceM[i][j] == 'x':
        tracer = 'x'
      elif traceM[i][j] == 'y':
        tracer = 'y'
      i, j = i - 1, j - 1
      continue
    
    elif tracer == 'x':
      aln1 = '-' + aln1
      aln2 = row[i] + aln2
      if traceIx[i][j] == 'm':
        tracer = 'm'    
      elif traceIx[i][j] == 'x':
        tracer = 'x'
      i = i - 1
      continue
      
    elif tracer == 'y':
      aln1 = col[j] + aln1
      aln2 = '-' + aln2
      if traceIy[i][j] == 'm':
        tracer = 'm'
      elif traceIy[i][j] == 'y':
        tracer = 'y'
      j = j - 1
    else:
      break
      
  return aln1, aln2

def isbest(a, b):
  if float(a) > float(b):
    return a
  else:
    return b

# Local Alignment  
def localAlignment3():
  bestScore = 0.0
  # Filling score matrices and keeping track of the direction for each cell
  M = np.zeros((len(row), len(col))) 
  Ix = np.zeros((len(row), len(col)))
  Iy = np.zeros((len(row), len(col)))

  traceM = np.zeros((len(row), len(col)), dtype = object)
  traceIx = np.zeros((len(row), len(col)), dtype = object)
  traceIy = np.zeros((len(row), len(col)), dtype = object)

  for i in range(1, len(row)):
    for j in range(1, len(col)):
      mat = score.loc[row[i]][col[j]]
      M[i][j] = max((M[i-1][j-1] + mat), (Ix[i-1][j-1] + mat), (Iy[i-1][j-1] + mat), 0.0) 
      Ix[i,j] = max((M[i - 1, j] + gop), (Ix[i - 1, j] + gex), 0.0)
      Iy[i,j] = max((M[i, j - 1] + gop), (Iy[i, j - 1] + gex), 0.0)

      if M[i][j] == (Ix[i-1][j-1] + mat):
        traceM[i][j] = 'x'
        bestScore = isbest(M[i][j], bestScore)
      elif M[i][j] == (Iy[i-1][j-1] + mat):
        traceM[i][j] = 'y'
        bestScore = isbest(M[i][j], bestScore)
      elif M[i][j] == (M[i-1][j-1] + mat):
        traceM[i][j] = 'm'
        bestScore = isbest(M[i][j], bestScore)

      if Ix[i][j] == (Ix[i - 1, j] + gex):
        traceIx[i][j] = 'x'
        bestScore = isbest(Ix[i][j], bestScore)
      elif Ix[i][j] == (M[i - 1, j] + gop):
        traceIx[i][j] = 'm'
        bestScore = isbest(Ix[i][j], bestScore)
      
      if Iy[i][j] == (Iy[i, j - 1] + gex):
        traceIy[i][j] = 'y'
        bestScore = isbest(Iy[i][j], bestScore)
      elif Iy[i][j] == (M[i, j - 1] + gop):
        traceIy[i][j] = 'm'   
        bestScore = isbest(Iy[i][j], bestScore)

# Trace back for local 
  r, c = M.shape # row and col lengths to track best score locations
  bestLoc = (0,0) 
  
  # Find  location of the best score 
  for i in range(1, r):
    for j in range(1, c):
      if M[i][j] == bestScore:
        bestLoc = (i, j)
      if Ix[i][j] == bestScore:
        bestLoc = (i, j)
      if Iy[i][j] == bestScore:
        bestLoc = (i, j)
  
  i, j = bestLoc
  aln1 = ''
  aln2 = ''
  mxm = max(M[i][j], Ix[i][j], Iy[i][j], 0)
  tracer = ''
  if mxm == M[i][j]:
    tracer = 'm'
  elif mxm == Ix[i][j]:
    tracer = 'x'
  else:
    tracer = 'y'
      
  while (i > 0 and j > 0):
    if tracer == 'm' and M[i][j]>0:
      aln1 = col[j] + aln1
      aln2 = row[i] + aln2
      if traceM[i][j] == 'm':
        tracer = 'm'
      elif traceM[i][j] == 'x':
        tracer = 'x'
      elif traceM[i][j] == 'y':
        tracer = 'y'
      i, j = i - 1, j - 1
      continue
      
    elif tracer == 'x' and Ix[i][j] > 0:
      aln1 = '-' + aln1
      aln2 = row[i] + aln2
      if traceIx[i][j] == 'm':
        tracer = 'm'    
      elif traceIx[i][j] == 'x':
        tracer = 'x'
      i = i - 1
      continue
        
    elif tracer == 'y' and Iy[i][j] > 0:
      aln1 = col[j] + aln1
      aln2 = '-' + aln2
      if traceIy[i][j] == 'm':
        tracer = 'm'
      elif traceIy[i][j] == 'y':
        tracer = 'y'
      j = j - 1
    else:
      break
  return aln1, aln2
#localAlignment3()

In [11]:
globalAlignment3()

('MRTLSPARPHRTPSKSLPPCSTPACSPRSSASTVFRSLSLLQSWILTLSLHLHRQRQLARVNCPVTMDFKRLIADLSPAQRAEVLASLIPMMSTTELQLARQLIRTPSGPITVPVAPVCHAHATLSMSWPGPAPSTVPRSKSPDCACACTCTDGTPRPQFRDLPDEILLKIVAYL-RPQSLARLARTCHRFAVLIRDSLLWKNMLAKYRGQGYAAYRPHEDKLRVLRSTTTPPAPIPIVYSPEKALVRRDSGVARSPTAAAALFGTSPDGSRNGSLPARNGHQEIPKSLGSFWRDEFRRGYLTHLNWKRGRCRIAKVLRRPGSLHADMDPDRNLIASIDADGSKTWDVMTGETTTQLPSQLGS-FTTIKFFQHYTLTGSTNCSVRVWDHATG-QLVRA----FTGVHTREVSAIDADQATVASGSEDTVIWTWEL---LS------GKPLQ--------------YFRGHTGPICSLKLDMARGILLSGSADTTVRVWDVAPTLM---APDNPTTPIVQLGKCRAVLRGHQNEVFCVALASNLVVSGSGDCRIKVWSL-----ETKS---------LLRTLEGHTAAVIALQVDNNKIISGSADKTIKVWDLHN----PAQALYSLSDQHTSGLLALRFNDKQLFASDMASTIVVYDFEHEDDKQDTDPRAPPSPSSSTGSMHFCTY',
 'M----------TPWDTVAP----------------------------QTLSLPRERRIA------------------------LLAALLRDASDSDLLLAQ----AQAVATLLPEIPT----ASLN-------GDGFQHKRLP--------------PLLMDLPDELLIHVLARIATPADLAATARTSVRMAAIVRDNHLWRDLVVRLGAEGFGAY----------------PRPPPVT--------RRDSGLL------AVVVASGPSETKTTKVP----RSMVRSP----WHSEFVKGYKTAMN

# globalAlignment2()

In [16]:

def globalAlignment2():
  M = np.zeros((len(row), len(col))) 
  Ix = np.zeros((len(row), len(col)))
  Iy = np.zeros((len(row), len(col)))

  traceM = np.zeros((len(row), len(col)), dtype = object)
  traceIx = np.zeros((len(row), len(col)), dtype = object)
  traceIy = np.zeros((len(row), len(col)), dtype = object)

  Ix[0][0] = Iy[0][0] = gop
  for i in range(1, len(row)):
    M[i][0] = Iy[i][0] = np.NINF
    Ix[i][0] = gop + i * gex 
  for j in range(1, len(col)):
    M[0][j] = Ix[0][j] = np.NINF
    Iy[0][j] = gop + j * gex

  for i in range(1, len(row)):
    for j in range(1, len(col)):
      mat = score.loc[row[i]][col[j]]
      M[i][j] = max((M[i-1][j-1] + mat), (Ix[i-1][j-1] + mat), (Iy[i-1][j-1] + mat)) 
      Ix[i,j] = max((M[i - 1, j] + gop), (Ix[i - 1, j] + gex))
      Iy[i,j] = max((M[i, j - 1] + gop), (Iy[i, j - 1] + gex))

      if M[i][j] == (Iy[i-1][j-1] + mat):
        traceM[i][j] = 'y'
      elif M[i][j] == (Ix[i-1][j-1] + mat):
        traceM[i][j] = 'x'
      else:
        traceM[i][j] = 'm'

      if Ix[i][j] == (Ix[i - 1, j] + gex):
        traceIx[i][j] = 'x'
      else:
        traceIx[i][j] = 'm'
      
      if Iy[i][j] == (Iy[i, j - 1] + gex):
        traceIy[i][j] = 'y'
      else:
        traceIy[i][j] = 'm'   

# Trace back  for global 
  j = len(seq1)
  i = len(seq2)
  aln1 = ''
  aln2 = ''
  mxm = max(M[i][j], Ix[i][j], Iy[i][j])
  tracer = ''
  if mxm == M[i][j]:
    tracer = 'm'
  elif mxm == Ix[i][j]:
    tracer = 'x'
  else:
    tracer = 'y'
    
  while (i > 0 and j > 0):
    if tracer == 'm':
      aln1 = col[j] + aln1
      aln2 = row[i] + aln2
      #i, j = i - 1, j - 1
      if traceM[i][j] == 'm':
        tracer = 'm'
      elif traceM[i][j] == 'x':
        tracer = 'x'
      elif traceM[i][j] == 'y':
        tracer = 'y'
      i, j = i - 1, j - 1
      #continue
    
    elif tracer == 'x':
      aln1 = '-' + aln1
      aln2 = row[i] + aln2
      #i = i - 1
      if traceIx[i][j] == 'm':
        tracer = 'm'    
      elif traceIx[i][j] == 'x':
        tracer = 'x'
      i = i - 1
      #continue
      
    elif tracer == 'y':
      aln1 = col[j] + aln1
      aln2 = '-' + aln2
      #j = j - 1
      if traceIy[i][j] == 'm':
        tracer = 'm'
      elif traceIy[i][j] == 'y':
        tracer = 'y'
      j = j - 1
    #else:
      #break
      
  print(M)
  return aln1, aln2
# globalAlignment2()

In [17]:
globalAlignment2()

[[    0.   -inf   -inf ...   -inf   -inf   -inf]
 [  -inf     6.   -12. ... -1281. -1279. -1282.]
 [  -inf   -13.     5. ... -1268. -1265. -1273.]
 ...
 [  -inf -1078. -1067. ...   471.   474.   469.]
 [  -inf -1079. -1069. ...   478.   481.   473.]
 [  -inf -1078. -1072. ...   480.   480.   479.]]


('MRTLSPARPHRTPSKSLPPCSTPACSPRSSASTVFRSLSLLQSWILTLSLHLHRQRQLARVNCPVTMDFKRLIADLSPAQRAEVLASLIPMMSTTELQLARQLIRTPSGPITVPVAPVCHAHATLSMSWPGPAPSTVPRSKSPDCACACTCTDGTPRPQFRDLPDEILLKIVAYL-RPQSLARLARTCHRFAVLIRDSLLWKNMLAKYRGQGYAAYRPHEDKLRVLRSTTTPPAPIPIVYSPEKALVRRDSGVARSPTAAAALFGTSPDGSRNGSLPARNGHQEIPKSLGSFWRDEFRRGYLTHLNWKRGRCRIAKVLRRPGSLHADMDPDRNLIASIDADGSKTWDVMTGETTTQLPSQLGS-FTTIKFFQHYTLTGSTNCSVRVWDHATG-QLVRA----FTGVHTREVSAIDADQATVASGSEDTVIWTWEL---LS------GKPLQ--------------YFRGHTGPICSLKLDMARGILLSGSADTTVRVWDVAPTLM---APDNPTTPIVQLGKCRAVLRGHQNEVFCVALASNLVVSGSGDCRIKVWSL-----ETKS---------LLRTLEGHTAAVIALQVDNNKIISGSADKTIKVWDLHN----PAQALYSLSDQHTSGLLALRFNDKQLFASDMASTIVVYDFEHEDDKQDTDPRAPPSPSSSTGSMHFCTY',
 'M----------TPWDTVAP----------------------------QTLSLPRERRIA------------------------LLAALLRDASDSDLLLAQ----AQAVATLLPEIPT----ASLN-------GDGFQHKRLP--------------PLLMDLPDELLIHVLARIATPADLAATARTSVRMAAIVRDNHLWRDLVVRLGAEGFGAY----------------PRPPPVT--------RRDSGLL------AVVVASGPSETKTTKVP----RSMVRSP----WHSEFVKGYKTAMN

In [36]:
print(seq11)
print(seq22)
print(sq2)
print(s2)

MRTLSPARPHRTPSKSLPPCSTPACSPRSSASTVFRSLSLLQSW--ILTLSLHLHRQRQLARVNCPVTMDFKRLIADLSPAQRAEVLASLIPMMSTTELQLARQLIRTPSGPITVPVAPVCHAHATLSMSWPGPAPSTVPRSKSPDCACACTCTDGTPRPQFRDLPDEILLKIVAYL-RPQSLARLARTCHRFAVLIRDSLLWKNMLAKYRGQGYAAYRPHEDKLRVLRSTTTPPAPIPIVYSPEKALVRRDSGVARSPTAAAALFGTSPDGSRNGSLPARNGHQEIPKSLGSFWRDEFRRGYLTHLNWKRGRCRIAKVLRRPGSLHADMDPDRNLIASIDADGSKTWDVMTGETTTQLPSQ-LGSFTTIKFFQHYTLTGSTNCSVRVWDHATG-QLVRAFTG----VHTREVSAIDADQATVASGSEDTVIWTWE----LL-----SGKPL--------------QYFRGHTGPICSLKLDMARGILLSGSADTTVRVWDVAPTL---MAPDNPTTPIVQLGKCRAVLRGHQNEVFCVALASNLVVSGSGDCRIKVW--------------SLETKSLLRTLEGHTAAVIALQVDNNKIISGSADKTIKVWDLH----NPAQALYSLSDQHTSGLLALRFNDKQLFASDMASTIVVYDFEHEDDKQDTDPRAPPSPSSSTGSMHFCTY
M----------------------------------------TPWDTVAPQTLSLPRER------------------------RIALLAALLRDASDSDLLLA----------QAQAVATLLPEIPTASLN-----GDGFQHKRLP--------------PLLMDLPDELLIHVLARIATPADLAATARTSVRMAAIVRDNHLWRDLVVRLGAEGFGAY----------------PRPPPVT--------RRDSGL------LAVVVASGPSETKTTKVP--------RSMVRSPWHSEFVKGYKTAMNWK

In [31]:
pairwise(s1, s2)

479

In [ ]:
# Expected Output 
c = "N-LFVALYDFVASGDNTLSITKGEKLRVLGYNHNGE--WCEAQTKNGQGWVPSNYITPVN"
d = "KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP"
print(pairwise(c,d))
# Another possible output with the same SoP score
c = "N-LFVALYDFVASGDNTLSITKGEKLRVLGYNHNG--EWCEAQTKNGQGWVPSNYITPVN"
d = "KGVIYALWDYEPQNDDELPMKEGDCMTIIHREDEDEIEWWWARLNDKEGYVPRNLLGLYP"
print(pairwise(c,d))

In [22]:
from pprint import pprint

global S
global E
global match
global mismatch
global MIN
S   = -10.0
E   = -2.0
MIN = -float("inf")

#initializers for matrices
def _init_x(i, j):
    if i > 0 and j == 0:
        return MIN
    else:
        if j > 0:
            return -10 + (-0.5 * j)
        else:
            return 0

def _init_y(i, j):
    if j > 0 and i == 0:
        return MIN
    else:
        if i > 0:
            return -10 + (-0.5 * i)
        else:
            return 0

def _init_m(i, j):
    if j == 0 and i == 0:
        return 0
    else:
        if j == 0 or i == 0:
            return MIN
        else:
            return 0

def _format_tuple(inlist, i, j):
    return 0

def distance_matrix(s, t):
    dim_i = len(t) + 1
    dim_j = len(s) + 1
    #abuse list comprehensions to create matrices
    X = [[_init_x(i, j) for j in range(0, dim_j)] for i in range(0, dim_i)]
    Y = [[_init_y(i, j) for j in range(0, dim_j)] for i in range(0, dim_i)]
    M = [[_init_m(i, j) for j in range(0, dim_j)] for i in range(0, dim_i)]

    for j in range(1, dim_j):
        for i in range(1, dim_i):
            X[i][j] = max((S + E + M[i][j-1]), (E + X[i][j-1]), (S + E + Y[i][j-1]))
            Y[i][j] = max((S + E + M[i-1][j]), (S + E + X[i-1][j]), (E + Y[i-1][j]))
            mat = score.loc[t[i-1]][s[j-1]]
            M[i][j] = max(mat + M[i-1][j-1], X[i][j], Y[i][j])
            #M[i][j] = max(_match(s, t, i, j) + M[i-1][j-1], X[i][j], Y[i][j])
            #print('forward ', i, j)

    #return [X, Y, M]
    return X, Y, M

#def backtrace(s, t, X, Y, M):
def backtrace(s, t):
    X, Y, M = distance_matrix(s, t)
    sequ1 = ''
    sequ2 = ''
    i = len(t)
    j = len(s)
    while (i>0 and j>0):
        mat = score.loc[t[i-1]][s[j-1]]
        #if (i>0 and j>0 and M[i][j] == M[i-1][j-1] + _match(s, t, i, j)):
        if (i>0 and j>0 and M[i][j] == M[i-1][j-1] + mat):
            sequ1 += s[j-1]
            sequ2 += t[i-1]
            i -= 1; j -= 1
        elif (i>0 and M[i][j] == Y[i][j]):
            sequ1 += '_'
            sequ2 += t[i-1]
            i -= 1
        elif (j>0 and M[i][j] == X[i][j]):
            sequ1 += s[j-1]
            sequ2 += '_'
            j -= 1
        print('Traceback ', i, j, t[i-1], s[j-1])

    sequ1r = ' '.join([sequ1[j] for j in range(-1, -(len(sequ1)+1), -1)])
    sequ2r = ' '.join([sequ2[j] for j in range(-1, -(len(sequ2)+1), -1)])

    #return [sequ1r, sequ2r]
    return sequ1r, sequ2r


In [23]:
backtrace(seq1, seq2)

Traceback  535 635 T T
Traceback  534 634 E C
Traceback  534 633 E F
Traceback  534 632 E H
Traceback  534 631 E M
Traceback  534 630 E S
Traceback  534 629 E G
Traceback  533 628 S T
Traceback  532 627 P S
Traceback  531 626 E S
Traceback  531 625 E S
Traceback  531 624 E P
Traceback  531 623 E S
Traceback  531 622 E P
Traceback  531 621 E P
Traceback  531 620 E A
Traceback  531 619 E R
Traceback  531 618 E P
Traceback  531 617 E D
Traceback  530 616 E T
Traceback  530 615 E D
Traceback  530 614 E Q
Traceback  530 613 E K
Traceback  530 612 E D
Traceback  530 611 E D
Traceback  530 610 E E
Traceback  529 609 F H
Traceback  529 608 F E
Traceback  529 607 F F
Traceback  528 606 D D
Traceback  527 605 Y Y
Traceback  526 604 V V
Traceback  525 603 V V
Traceback  524 602 V I
Traceback  523 601 A T
Traceback  522 600 S S
Traceback  521 599 T A
Traceback  520 598 F M
Traceback  519 597 D D
Traceback  518 596 S S
Traceback  517 595 A A
Traceback  516 594 I F
Traceback  515 593 L L
Traceback  

('_ A Q R A E V L A S L I P M M S T T E L Q L A R Q L I R T P S G P I T V P V A P V C H A H A T L S M S W P G P A P S T V P R S K S P D C A C A C T C T D G T P R P Q F R D L P D E I L L K I V A Y L _ R P Q S L A R L A R T C H R F A V L I R D S L L W K N M L A K Y R G Q G Y A A Y R P H E D K L R V L R S T T T P P A P I P I V Y S P E K A L V R R D S G V A R S P T A A A A L F G T S P D G S R N G S L P A R N G H Q E I P K S L G S F W R D E F R R G Y L T H L N W K R G R C R I A K V L R R P G S L H A D M D P D R N L I A S I D A D G S K T W D V M T G E T T T Q L P S Q _ L G S F T T I K F F Q H Y T L T G S T N C S V R V W D H A T G _ Q L V _ _ R _ _ A F T G V H T R E V S A I D A D Q A T V A S G S E D T V I W T W E _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ L _ _ _ _ _ _ _ L S G _ K P L Q Y F R G H T G P I C S L K L D M A R G I L L S G S A D T T V R V W D V A P T L _ _ M _ A P D N P T T P I V Q L G K C R A V L R G H Q N E V F C V A L A S N L V V S G S G D C R I K V W _ _ _ _ _ _ _ _ _ _ _ S L _ _ _ E T K S 